In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import onnxruntime as ort
import onnx, os, torch, tf2onnx
from onnx2pytorch import ConvertModel
import matplotlib.pyplot as plt
import torch.nn as nn

import warnings

In [ ]:

sess = ort.InferenceSession('PipeSeg_tiny.onnx')
inp, out = sess.get_inputs()[0], sess.get_outputs()[0]

In [ ]:
video = cv2.VideoCapture('/content/drive/MyDrive/PipeSeg/DJI_0463.MP4')


In [ ]:
import cv2


# Create an object to read
# from camera
video = cv2.VideoCapture('/content/drive/MyDrive/PipeSeg/DJI_0463.MP4')

# We need to check if camera
# is opened previously or not
if (video.isOpened() == False):
	print("Error reading video file")

# We need to set resolutions.
# so, convert them from float to integer.
frame_width = int(video.get(3))
frame_height = int(video.get(4))

size = (frame_width, frame_height)

# Below VideoWriter object will create
# a frame of above defined The output
# is stored in 'filename.avi' file.
result = cv2.VideoWriter('output.avi',
						cv2.VideoWriter_fourcc(*'MJPG'),
						30, size)

c = 0
while(True):
	ret, frame = video.read()

	if ret == True:


		if c % 500 != 0:
			c+=1
			continue

		print(c)
		frame = cv2.resize(frame, (320, 240)) ## Resize your images according to your model input
		frame = frame[:,:,::-1]/255.0 ## If your traning is done on the normalized image otherwise comment this line
		plt.imshow(frame)
		plt.show()
		frame = frame.reshape(1, 240, 320, 3).astype(np.float32) 

		# pred = m.predict(frame)

		pred = sess.run([out.name], {inp.name: frame})[0].reshape(240, 320)

		pred = np.where(pred>=0.5, 0, 1)*255 #.astype(np.uint8)
		pred = pred.reshape(240, 320).astype(np.uint8)
		pred = np.dstack([pred, pred, pred]) ## not necessary,  here we did because we want to stack output(binary) of model with input(RGB) image
		# print(np.unique(pred))
		# result.write(pred)

		plt.imshow(pred)
		plt.show()

		c += 1

		# Press S on keyboard to stop the process
		if cv2.waitKey(1) & 0xFF == ord('s'):
			break



	# Break the loop
	else:
		break

video.release()
result.release()

cv2.destroyAllWindows()

print("The video was successfully saved")